In [1]:
from quantum_dataset import QM7, QM7b, QM9, Champs, SuperSet
from quantum_learning import Learn, Selector, ChampSelector
from quantum_model import FFNet
from torch.optim import Adam
from torch.nn import MSELoss, L1Loss

RDKit WARNING: [18:03:23] Enabling RDKit 2019.09.3 jupyter extensions


In [ ]:
opt_params = {'lr': 0.001}
model_params = {'D_in': 29*29+29+15+128+32+64+32+64, 'H': 8192, 'D_out': 1, 'model_name': 'funnel'}
qm9_params = {'n': 133885, 
              'features': ['coulomb','mulliken','A','B','C','mu','alpha','homo',
                           'lumo','gap','r2','zpve','U0','U','H','G','Cv'],  
              'dim': 29, 
              'use_pickle': True}
champs_params = {'n': 4658146, 
                 'features': False,
                 'use_h5': False,  
                 'infer': False}
crit_params = {'reduction': 'sum'}
ds_params = {'PrimaryDS': Champs, 'SecondaryDS': QM9, 'p_params': champs_params, 's_params': qm9_params}
sample_params = {'split': .1, 'subset': False}

Learn(Dataset=SuperSet, Model=FFNet, Sampler=ChampSelector, Optimizer=Adam, Criterion=L1Loss,
      batch_size=32, epochs=3, model_params=model_params, ds_params=ds_params, 
      sample_params=sample_params, opt_params=opt_params, crit_params=crit_params, 
      save_model=True, load_model=False)

In [2]:
opt_params = {'lr': 0.001}
model_params = {'D_in': 29*29+29+15+128+32+64+32+64, 'H': 8192, 'D_out': 1, 'model_name': 'funnel'}
qm9_params = {'n': 133885, 
              'features': ['coulomb','mulliken','A','B','C','mu','alpha','homo',
                           'lumo','gap','r2','zpve','U0','U','H','G','Cv'],  
              'dim': 29, 
              'use_pickle': True}
champs_params = {'n': 2505542, 
                 'features': False,
                 'use_h5': False,  
                 'infer': True}
crit_params = {'reduction': 'sum'}
ds_params = {'PrimaryDS': Champs, 'SecondaryDS': QM9, 'p_params': champs_params, 's_params': qm9_params}

Learn(Dataset=SuperSet, Model=FFNet, Sampler=ChampSelector, batch_size=32, 
      model_params=model_params, ds_params=ds_params, 
      load_model='./models/20200629_1800.pth', adapt=False)

loading QM9 datadic from a pickled copy...
<class 'quantum_dataset.SuperSet'> dataset created...
model loaded from state_dict...
self.predictions.iloc[:10]         id  scalar_coupling_constant
0  4957831                  1.698278
1  5539157                  9.207827
2  5144079                 48.540695
3  5680997                  0.745624
4  4889680                 81.590790
5  5878362                  1.940133
6  5367937                 55.580872
7  5603920                 -1.071535
8  4997250                 -1.980003
9  5560962                 82.959160
inference complete and saved to csv...
learning time: 0:11:29.363401


In [ ]:
model_params = {'D_in': 29*29+29+15+128+32+64+32+64, 
                'H': 8192, 
                'D_out': 1, 
                'model_name': 'funnel'}
ds_params = {'n': 2505542, 
             'features': False,
             'use_h5': False,  
             'infer': True}

Learn(Dataset=Champs, Model=FFNet, Sampler=Selector, batch_size=32, model_params=model_params, 
      ds_params=ds_params, load_model='./models/20200629_1800.pth', adapt=False)

In [ ]:
#create an inference baseline result using the mean training values for each type of scc
df = pd.read_csv('./data/champs/'+'train.csv', header=0, names=['id','molecule_name', 
                 'atom_index_0','atom_index_1','type','scalar_coupling_constant'], index_col=False)
dt = df.groupby('type')['scalar_coupling_constant'].describe()
test = pd.read_csv('./data/champs/'+'test.csv', header=0, names=['id','molecule_name', 
                   'atom_index_0','atom_index_1','type'], index_col=False)
test['scalar_coupling_constant'] = [dt['mean'].loc[x] for x in test['type']]
test[['id','scalar_coupling_constant']].to_csv('mean_inference.csv', 
                                               header=['id','scalar_coupling_constant'], 
                                               index=False)